[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/sort_patient_images.ipynb)

# **患者リストから甲状腺眼症を重症度別に抽出**

In [ ]:
import os
from PIL import *
import pandas as pd

In [ ]:
folder_path = "/Volumes/Extreme SSD/眼位写真NEW"
csv_path = "/Volumes/Extreme SSD/Disease_extracted_甲状腺眼症.バセドウ病眼症.csv"


def openCSV(csv_path):
    #df = pd.read_csv(csv_path, encoding="shift-jis")
    df = pd.read_csv(csv_path, encoding="utf-8", header=None)
    #df = df.drop(df.columns[[0]], axis=1) #1列目を削除する
    #print(df)
    return df

df = openCSV(csv_path)
df

In [ ]:
#甲状腺眼症から始まるものを抽出
df_temp = df.dropna(subset=[8])
df_GO = df_temp[df_temp[8].str.startswith('甲状腺眼症')]

#中等症以上ものを抽出
df_temp = df.dropna(subset=[8])
df_moderate = df_temp[df_temp[8].str.contains('中等症')]

#ステロイドありを抽出
df_temp = df.dropna(subset=[9])
df_steroids = df_temp[df_temp[9].str.contains('ステロイドあり')]

#中等症以上とステロイドありを合算
df_treatable = pd.concat([df_moderate, df_steroids]).drop_duplicates()


GO_list = df_treatable[2].tolist()
GO_list

In [ ]:
os.path.basename(glob.glob(folder_path +"/*")[0])

'1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554.jpg'

In [ ]:
import os
import glob
import shutil
from tqdm import tqdm


k=1
for path in tqdm(glob.glob(folder_path +"/*")):
    #画像IDがGO_listから始まるものを抜き出す
    #print(os.path.basename(path).rsplit("-")[0])
    if int(os.path.basename(path).rsplit("-")[0]) in GO_list:
        shutil.copy(path, f"/Volumes/Extreme SSD/Treatable_GO_images/{os.path.basename(path)}")
        k+=1

print(f"total {k} files picked up!")


100%|██████████| 79905/79905 [13:54<00:00, 95.74it/s]  

total 4432 files picked up!


#**オリンピアデータセットの抽出**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import shutil
import glob

full_dataset_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset" #編集未の画像を格納されているフォルダ
reference_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_500px" #治療された患者のIDが格納されているフォルダ

# reference_path内のファイルのベース名を取得
reference_files = [os.path.basename(file) for file in glob.glob(os.path.join(reference_path, "*"))]

# full_dataset_path内のファイルと比較して一致するファイルのパスを取得
matching_files = [file for file in glob.glob(os.path.join(full_dataset_path, "*")) if os.path.basename(file) in reference_files]

# 最初の5つのファイルパスを表示
for file_path in matching_files[:5]:
    print(file_path)


/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/19.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/28.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/30.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/35.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/36.JPG


In [15]:
#img2square
import os
import shutil
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from PIL import Image

def convert(matching_files, out_path):
    start = time.time()

    for i in matching_files:
          img = Image.open(i)
          # "olympia_"をファイル名の先頭に追加
          new_filename = "olympia_" + os.path.basename(i)
          img_new = expand2square(img, (0, 0, 0)).resize((768, 768))
          img_new.save(os.path.join(out_path, new_filename))
          print(os.path.join(out_path, new_filename))

    print('Process done!!')
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width-height)//2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


In [ ]:
#画像出力
out_path = '/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav333'
if not os.path.exists(out_path):
    os.makedirs(out_path)

convert(matching_files, out_path)